<https://github.com/veltman/presidential-election-results/blob/master/data.csv>

In [1]:
import pandas as pd
import numpy as np

#stuff for plotting states
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import matplotlib.colors as mcolors
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Polygon
import missingno as msno
import os
import openpyxl
import math

In [2]:
df = pd.read_csv('../data/pres_elec_res.csv')
df = df[df['year'].isin([2012, 2016, 2020])].iloc[:, :6]
df.index = np.arange(1, len(df) + 1)
df.head()

,year,state,demVotes,gopVotes,otherVotes,totalVotes
1,2012,AK,122640,164676,13179,300495
2,2016,AK,116454,163387,38767,318608
3,2020,AK,153778,189951,15801,359530
4,2012,AL,795696,1255925,22717,2074338
5,2016,AL,729547,1318255,75570,2123372


In [3]:
state_mapping = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming',
    'DC': 'District of Columbia'
}

In [4]:
mod_df = pd.DataFrame(df['state'].unique(), columns=['state'])
for year in [2012, 2016, 2020]:
    specific_year = year
    # Create Series for demVotes
    demVotes_year = (df.apply(lambda row: row['demVotes'] if row['year'] == specific_year else None, axis=1)).dropna().reset_index(drop=True)
    demVotes_year_df = pd.DataFrame(demVotes_year, columns=[f'demVotes_{specific_year}'])
    
    # Create Series for gopVotes
    gopVotes_year = (df.apply(lambda row: row['gopVotes'] if row['year'] == specific_year else None, axis=1)).dropna().reset_index(drop=True)
    gopVotes_year_df = pd.DataFrame(gopVotes_year, columns=[f'gopVotes_{specific_year}'])
    
    # Create Series for otherVotes
    otherVotes_year = (df.apply(lambda row: row['otherVotes'] if row['year'] == specific_year else None, axis=1)).dropna().reset_index(drop=True)
    otherVotes_year_df = pd.DataFrame(otherVotes_year, columns=[f'otherVotes_{specific_year}'])
    
    # Create Series for totalVotes
    totalVotes_year = (df.apply(lambda row: row['totalVotes'] if row['year'] == specific_year else None, axis=1)).dropna().reset_index(drop=True)
    totalVotes_year_df = pd.DataFrame(totalVotes_year, columns=[f'totalVotes_{specific_year}'])

    mod_df = pd.concat([mod_df, demVotes_year_df, gopVotes_year_df, otherVotes_year_df, totalVotes_year_df], axis=1)
mod_df['state'] = mod_df['state'].replace(state_mapping)
mod_df = mod_df.rename(columns={'state': 'Region'})
mod_df.head()

,Region,demVotes_2012,gopVotes_2012,otherVotes_2012,totalVotes_2012,demVotes_2016,gopVotes_2016,otherVotes_2016,totalVotes_2016,demVotes_2020,gopVotes_2020,otherVotes_2020,totalVotes_2020
0,Alaska,122640.0,164676.0,13179.0,300495.0,116454.0,163387.0,38767.0,318608.0,153778.0,189951.0,15801.0,359530.0
1,Alabama,795696.0,1255925.0,22717.0,2074338.0,729547.0,1318255.0,75570.0,2123372.0,849648.0,1441168.0,32488.0,2323304.0
2,Arkansas,394409.0,647744.0,27315.0,1069468.0,380494.0,684872.0,65269.0,1130635.0,423932.0,760647.0,34490.0,1219069.0
3,Arizona,1025232.0,1233654.0,40368.0,2299254.0,1161209.0,1252401.0,159555.0,2573165.0,1672143.0,1661686.0,53497.0,3387326.0
4,California,7854285.0,4839958.0,344304.0,13038547.0,8753788.0,4483810.0,943997.0,14181595.0,11109764.0,6005961.0,380181.0,17495906.0


In [5]:
mod_df.to_csv(os.path.join('../output/presidential','table.csv'))